## Importing the relevant packages

In [1]:
import sys
print(sys.path)
print(sys.executable)
!which python
!which ipython
!which jupyter

['', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/apurva/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/apurva/.ipython']
/usr/bin/python3
/home/apurva/anaconda3/envs/bbopt/bin/python
/home/apurva/anaconda3/envs/bbopt/bin/ipython
/home/apurva/anaconda3/envs/bbopt/bin/jupyter


In [2]:
import numpy as np
import pandas as pd
# %load_ext autoreload
# %autoreload 2
from sklearn.svm import SVC,SVR
import os
import sys
from MFTreeSearchCV.MFTreeSearchCV import *
from mf.mf_func import *


## Just listing the contents of the main code directory

In [3]:
%ls MFTreeSearchCV/
%ls Falsification/

MFHOO.py           MFTreeSearchCV.py  __pycache__/
MFTreeFunction.py  __init__.py        converters.py
__init__.py  temp.ipynb


In [4]:
import numpy as np
import queue
from mf.mf_func import MFOptFunction
from utils.general_utils import map_to_cube
import sys
from examples.synthetic_functions import *
from mf.mf_func import get_noisy_mfof_from_mfof
import time
from MFTreeSearchCV.MFHOO import *

import examples.synthetic_functions

In [5]:
def branin_function(x, a, b, c, r, s, t):
    # """ Computes the Branin function. """
    x1 = x[0]
    x2 = x[1]
    neg_ret = a * (x2 - b*x1**2 + c*x1 - r)**2 + s*(1-t)*np.cos(x1) + s
    return -neg_ret

def branin_function_alpha(x, alpha, a, r, s):
    #""" Alternative form for the branin function. """
    return branin_function(x, a, alpha[0], alpha[1], r, s, alpha[2])

def get_mf_branin_function(fidel_dim):
    #""" Returns the Branin function as a multifidelity function. """
    a0 = 1
    b0 = 5.1/(4*np.pi**2)
    c0 = 5/np.pi
    r0 = 6
    s0 = 10
    t0 = 1/(8*np.pi)
    alpha = np.array([b0, c0, t0])
    # Define delta
    delta = [0.01, 0.1, -0.005]
    delta = np.array(delta[0:fidel_dim] + [0] * (3 - fidel_dim))
    
    def mf_branin_obj(z, x):
    #""" Wrapper for the MF Branin objective. """
        assert len(z) == fidel_dim
        z_extended = np.append(z, [0] * (3-fidel_dim))
        alpha_z = alpha - (1 - z_extended) * delta
        print(alpha_z)
        return branin_function_alpha(x, alpha_z, a0, r0, s0)
    
    # Other data
    opt_fidel = np.ones((fidel_dim))
    fidel_bounds = [[0, 1]] * fidel_dim
    opt_pt = np.array([np.pi, 2.275])
    domain_bounds = [[-5, 10], [0, 15]]
    return mf_branin_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds
    
def get_mf_branin_as_mfof(fidel_dim):
#""" Wrapper for get_mf_branin_function which returns as a mfof. """
    mf_branin_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds = get_mf_branin_function(fidel_dim)
    fidel_cost_function = _get_mf_cost_function(fidel_bounds, True)
    opt_val = mf_branin_obj(opt_fidel, opt_pt)
    return MFOptFunction(mf_branin_obj, fidel_cost_function, fidel_bounds, domain_bounds,
                       opt_fidel, vectorised=False, opt_pt=opt_pt, opt_val=opt_val)

In [6]:
# Testing Brannin Function:
fidel_dim = 1
mf_branin_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds = get_mf_branin_function(fidel_dim)
delta = [0.01, 0.1, -0.005]
delta = np.array(delta[0:fidel_dim] + [0] * (3 - fidel_dim))
mf_branin_obj([0.1], [3,1])
mf_branin_obj([0.9], [3,1])

[0.12018451 1.59154943 0.03978874]
[0.12818451 1.59154943 0.03978874]


-2.3956554273396637

In [7]:
NUM_EXP = 5
EXP_NAME = 'Branin'
#EXP_NAME = 'CurrinExp'
#EXP_NAME = 'Hartmann3'
#EXP_NAME = 'Hartmann6'
#EXP_NAME = 'Borehole'


def run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filname):
	R = []
	T = []
	for t in times:
		budget = t*mfobject.opt_fidel_cost
		t1 = time.time()
		MP = MFPOO(mfobject=mfobject, nu_max=nu, rho_max=rho, total_budget=budget, sigma=sigma, C=C, mult=0.5, tol = 1e-3, Randomize = False, Auto = True, unit_cost=t0 )
		MP.run_all_MFHOO()
		X, E = MP.get_point()
		t2 = time.time()	

		R = R + [E]
		T = T + [MP.cost]
		print(str(MP.cost) + ' : ' + str(E))
		#print 'Total HOO Queries: ' + str(MP.t) 

	np.save(filename,R)
	return np.array(R),np.array(T)


In [8]:
if EXP_NAME == 'Hartmann3':
    mfof = examples.synthetic_functions.get_mf_hartmann_as_mfof(1, 3)
    noise_var = 0.01
    sigma = np.sqrt(noise_var)

elif EXP_NAME == 'Hartmann6':
    mfof = examples.synthetic_functions.get_mf_hartmann_as_mfof(1, 6)
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.05
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'CurrinExp':
    mfof = examples.synthetic_functions.get_mf_currin_exp_as_mfof()
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.5
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'Branin':
    mfof = examples.synthetic_functions.get_mf_branin_as_mfof(1)
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.05
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'Borehole':
    mfof = examples.synthetic_functions.get_mf_borehole_as_mfof()
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 5
    sigma = np.sqrt(noise_var)

In [ ]:
times = [10,20,50,75,100,150,175, 200]
mfobject = get_noisy_mfof_from_mfof(mfof, noise_var)
nu = 1.0
rho = 0.95
C = 0.1
t0 = mfobject.opt_fidel_costmfobject = get_noisy_mfof_from_mfof(mfof, noise_var)
nu = 1.0
rho = 0.95
C = 0.1
t0 = mfobject.opt_fidel_cost

NT = str(time.time())
print('Running Experiment 1: ')
filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + '1.npy'
R,T = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
result = R

for i in range(1,NUM_EXP):
    print('Running Experiment' + str(i+1) + ': ')
    filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + str(i+1) + '.npy'
    R,T = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
    result = np.vstack([result,R])

mu = np.mean(result,axis = 0)
std = np.std(result,axis = 0)
result = mfobject.opt_val - mu
filename = './examples/results/MFHOO_' + EXP_NAME + '_' + NT + '_' + '.csv'
dfdic = {}
dfdic['Capital'] = np.array(times)
dfdic['Value'] = result
dfdic['Std'] = std
df = pd.DataFrame(dfdic)
df.to_csv(filename) 

NT = str(time.time())
print('Running Experiment 1: ')
filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + '1.npy'
R,T = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
result = R

for i in range(1,NUM_EXP):
    print('Running Experiment' + str(i+1) + ': ')
    filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + str(i+1) + '.npy'
    R,T = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
    result = np.vstack([result,R])


mu = np.mean(result,axis = 0)
std = np.std(result,axis = 0)
result = mfobject.opt_val - mu
filename = './examples/results/MFHOO_' + EXP_NAME + '_' + NT + '_' + '.csv'
dfdic = {}
dfdic['Capital'] = np.array(times)
dfdic['Value'] = result
dfdic['Std'] = std
df = pd.DataFrame(dfdic)
df.to_csv(filename) 

Running Experiment 1: 
Auto Init: 
C: 0.8313313703582592
nu: 0.8313313703582592
Budget Remaining: 9.253777793884279
Number of MFHOO Instances: 5
Budget per MFHOO Instance:0.8507555587768557
Running SOO number: 1 rho: 0.95 nu: 0.8313313703582592
Done!
Running SOO number: 2 rho: 0.9378956176563621 nu: 0.8313313703582592
Done!
Updating C
C: 1.163863918501563
nu_max: 1.163863918501563
Running SOO number: 3 rho: 0.9180634032924686 nu: 1.163863918501563
Done!
Updating C
C: 1.629409485902188
nu_max: 1.629409485902188
Running SOO number: 4 rho: 0.8796481896190089 nu: 1.629409485902188
Done!
Updating C
C: 2.281173280263063
nu_max: 2.281173280263063
Running SOO number: 5 rho: 0.7737809374999998 nu: 2.281173280263063
Done!
Updating C
C: 3.193642592368288
nu_max: 3.193642592368288
9.330126285552979 : [-0.5405371727128916, -0.5405296519642988, -0.5405371727128916, -0.5405214901684072, -0.5403576805741608]
Auto Init: 
C: 0.8313313703582592
nu: 0.8313313703582592
Budget Remaining: 19.25390653991699
N

## Setting up Branin function experiment
- Definitions of Brannin function given above. 


## Fetching mountain car dataset
- We will use the RSV dataset for the mountain car example 

## Constructing approximations of the Robust Satisfaction Value function
- Deriving the training data for mountain car

In [ ]:
estimator = LogisticRegression() #base estimator
 param_dict = {'C' : {'range': [1e-2,1e2], 'type': 'real', 'scale': 'linear'}, \
               'kernel' : {'range': [ 'linear', 'poly', 'rbf', 'sigmoid'], 'type': 'cat'}
param_dict = {'C':{'range':[1e-5,1e5],'scale':'log','type':'real'},\
              'penalty':{'range':['l1','l2'],'scale':'linear','type':'cat'}} #parameter space
fidelity_range = [500,15076] # fidelity range, lowest fidelity uses 500 samples while the highest one uses 
#the whole dataset  
n_jobs = 3 # number of jobs
cv = 3 # cv level
fixed_params = {}
scoring = 'accuracy'

## Budget
- We set the total budget as 100 secs
- This is may be only 3-4 times the budget required to do one single training and CV using the whole data-set

In [ ]:
t1 = time.time()
estimator = estimator.fit(X_train,y_train)
t2 = time.time()
total_budget = 100 # total budget in seconds
print('Time without CV: ', t2 - t1)

## Creating an instance of the class

In [ ]:
model = MFTreeSearchCV(estimator=estimator,param_dict=param_dict,scoring=scoring,\
                      fidelity_range=fidelity_range,unit_cost=None,\
                    cv=cv,  n_jobs = n_jobs,total_budget=total_budget,debug = True,fixed_params=fixed_params)

## running in debug mode will display certain outputs

## Fitting the model or choosing the best parameter
- Note that refit = true, which means at the end the training is done at the best parameter

In [ ]:
m = model.fit(X_train,y_train)

## Predicting using the best model, then scoring it and then displaying the best_params_

In [ ]:
y_pred = m.predict(X_test)

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
m.best_params_

In [ ]:
m.cv_results_